In [100]:
file_path = "data\Feinkonzept_NBG_final.pdf"

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:1: SyntaxWarning: invalid escape sequence '\F'
C:\Users\nicola.rudow\AppData\Local\Temp\ipykernel_1704\648987946.py:1: SyntaxWarning: invalid escape sequence '\F'
  file_path = "data\Feinkonzept_NBG_final.pdf"


In [90]:
import pdfplumber
import re
import os
from PyPDF2 import PdfReader, PdfWriter

In [109]:

def extract_text_and_toc_end_page(file_path):
    def is_toc_page(text):
        lines = text.splitlines()
        toc_like_lines = [line for line in lines if "..." in line or re.match(r'.*\d+$', line.strip())]
        return len(toc_like_lines) > len(lines) * 0.5

    def is_significant_content(text):
        lines = text.splitlines()
        long_lines = [line for line in lines if len(line) > 50]
        return len(long_lines) > len(lines) * 0.5

    with pdfplumber.open(file_path) as pdf:
        toc_text = ""
        toc_end_page = None

        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text and "Inhalt" in text:
                toc_text += text.split("Inhalt", 1)[1]

                for j in range(i + 1, len(pdf.pages)):
                    next_page_text = pdf.pages[j].extract_text()
                    if next_page_text:
                        if is_significant_content(next_page_text):
                            toc_end_page = j
                            break
                        if not is_toc_page(next_page_text):
                            toc_end_page = j
                            break
                        toc_text += next_page_text
                    else:
                        toc_end_page = j
                        break

                if toc_end_page is None:
                    toc_end_page = len(pdf.pages) - 1

                return toc_text, toc_end_page

    return None, None


In [110]:
text, end_page = extract_text_and_toc_end_page(file_path)

In [105]:
#print(end_page) #entkommentieren um Ende zu testen

2


In [ ]:
# Regex für den ersten Abschnitt
#first_pattern = r"^(?:(1(?:\.\d+)*\.?|AP 1(?:\.\d+)*\.?|AP 0\.?)(?:[.:]?)\s+([^\.\n]+))"


In [103]:
#Extrahieren der Abschnittsnamen im Inhaltsverzeichnis
# Regex für den ersten Abschnitt
first_pattern = r"^(?:(1(?:\.\d+)*\.?|AP 1(?:\.\d+)*\.?|AP 0\.?)(?:[.:]?)\s+([^\.\n]+))"

# Regex für alle weiteren Abschnitte
subsequent_pattern = r"^(?:(\d+\.\d*|AP \d+\.\d*|AP \d+:[.:]?|\d+)[.:]?)\s+([^\.\n]+)"

# Suche nach dem ersten Abschnitt
first_match = re.search(first_pattern, text, re.MULTILINE)

# Wenn der erste Match gefunden wurde, verarbeite die restlichen
if first_match:
    abschnitte = [f"{first_match.group(1).strip()} {first_match.group(2).strip()}"]
    
    # Schneide den Text ab dem ersten Match ab und suche nach den weiteren Abschnitten
    remaining_text = text[first_match.end():]
    subsequent_matches = re.findall(subsequent_pattern, remaining_text, re.MULTILINE)
    
    # Füge die restlichen Abschnitte hinzu
    abschnitte.extend([f"{num.strip()} {name.strip()}" for num, name in subsequent_matches])
else:
    abschnitte = []  # Falls kein gültiger erster Abschnitt gefunden wird

In [115]:

titles = []

for line in text:
    # Regex, um Titel zu finden, die entweder Punkte oder nur Seitenzahlen enthalten
    match = re.match(r'^(.*?)(\s+\.\.\.\s+\d+|\s+\d+)$', line.strip())
    if match:
        titles.append(match.group(1).strip())

# Ausgabe der extrahierten Titel
for title in titles:
    print(title)

In [ ]:
inhalt = text

# Liste für die Titel der Abschnitte
titel_liste = []

# Durchlaufe jede Zeile im Inhaltsverzeichnis
for zeile in inhalt:
    # Verwende einen regulären Ausdruck, um den Titel zu extrahieren
    titel = re.sub(r'\s*\.{3,}\s*\d+\s*$', '', zeile).strip()
    titel_liste.append(titel)

# Ausgabe der Titel
for titel in titel_liste:
    print(titel)

In [ ]:


# Liste für die Titel der Abschnitte
titel_liste = []

# Durchlaufe jede Zeile im Inhaltsverzeichnis
for zeile in inhalt:
    # Verwende einen regulären Ausdruck, um den Titel zu extrahieren
    titel = re.sub(r'\s*\d+\s*$', '', zeile).strip()  # Entfernt nur die Seitenzahl
    if titel:  # Stelle sicher, dass der Titel nicht leer ist
        titel_liste.append(titel)

# Ausgabe der Titel
for titel in titel_liste:
    print(titel)

In [104]:
print(abschnitte) #entkommentieren falls man testen will was gefunden wurde

[]


In [68]:
###PDF Aufräumen und Abschnitte extrahieren###

#Funktion um alles bis nach dem Innhaltsverzeichnis aus der pdf datei zu löschen, damit Inhaltsverzeichnis nicht erneut extrahiert wird
def remove_pages_before_contents(pdf_path, output_path, contents_start_page=end_page):
    """
    Remove pages before the content starts (after the table of contents).
    
    :param pdf_path: Path to the original PDF file.
    :param output_path: Path to save the new PDF file without the first pages.
    :param contents_start_page: The page number (0-indexed) where the content starts.
    :return: None
    """
    reader = PdfReader(pdf_path)
    writer = PdfWriter()

    # Add pages starting from the contents_start_page to the new PDF
    for page_num in range(contents_start_page, len(reader.pages)):
        writer.add_page(reader.pages[page_num])

    # Write the new PDF to the output path
    with open(output_path, "wb") as output_pdf:
        writer.write(output_pdf)

    print(f"New PDF saved to: {output_path}")

#Funktion um die einzelnen Abschnitte zu finden und zu extrahieren
def extract_sections_from_pdf(new_pdf_path, sections):
    """
    Extract specified sections from a PDF file, starting at the match's start.

    :param pdf_path: Path to the PDF file.
    :param sections: List of section headings to extract.
    :return: Dictionary with section headings as keys and extracted text as values.
    """
    # Read the PDF
    reader = PdfReader(pdf_path)
    text = ""
    
    # Combine text from all pages
    for page in reader.pages:
        text += page.extract_text()

    # Prepare a regex pattern to match sections
    sections_pattern = [re.escape(sec).replace(r'\ ', r'\s+') for sec in sections]
    regex = re.compile(f"({'|'.join(sections_pattern)})", re.MULTILINE)


    # Find matches and extract content
    matches = list(regex.finditer(text))
    extracted_sections = {}


    # Loop through the matches to extract text for each section
    for i, match in enumerate(matches):
        start = match.start()  # Start from the beginning of the match

        # Define the end position as the start of the next section or the end of the text
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        # Extract the text for this section
        section_text = text[start:end].strip()

        # Store the extracted section text
        extracted_sections[match.group()] = section_text
        print(extracted_sections)

    return extracted_sections

pdf_path = file_path
new_pdf_path = "data/cleaned_pdf.pdf"  # New PDF after removing pages up to the contents
sections = abschnitte  # Liste der Abschnitte, die du extrahieren möchtest

In [7]:
# Call Funktion to remove pages before the table of contents
remove_pages_before_contents(pdf_path, new_pdf_path) 

New PDF saved to: data/cleaned_pdf.pdf


In [52]:
# Extract sections from the cleaned PDF
extracted_sections = extract_sections_from_pdf(new_pdf_path, abschnitte)

In [55]:
reader = PdfReader(new_pdf_path)
text = ""

In [61]:
for page in reader.pages:
        text += page.extract_text()

In [62]:
text

' \n \n \n \n \n© Kantar /nexus  2022   4 \n \n1 Projekthintergrund und Zielsetzung  \n1.1 PROJEKTHINTERGRUND  \n5G ist die sogenannte fünfte Generation (daher 5G) der Netztechnologie in der mobilen Datenkommunikation, \nwelche gegenüber dem in Deutschland derzeit noch vorherrschenden 4G -Netz eine um mehrere Dimensionen \ngrößere Datenmenge transportieren kann. Das erste Mobilfunknetz in Deutschland, das A -Netz wurde 1958 \nvon der Bundespost eingerichtet. Es folgten das B -Netz (1972 -94), das C -Netz (1985 -2002), das D -Netz (seit \n1992) und das E -Netz (seit 1994). B - und C -Netz werden heute international als G1 -Netz bezeichnet, das D - \nund E -Netz als G2 -Netz. Erst mit dem neuen UM TS-Netz (G3) ab 2006 konnte sich der Mobilfunk zu den heute \nbekannten Smartphones mit einer Fülle von Funktionen über das Telefonieren hinaus entwickeln. Das LTE -\nNetz (G4), eingeführt 2010, steigerte die mögliche Datenmenge im Datenaustausch massiv, seit 2 019 folgt \ndas G5 -Netz. An Netz

In [63]:
sections_pattern = [re.escape(sec).replace(r'\ ', r'\s+') for sec in sections]
regex = re.compile(f"({'|'.join(sections_pattern)})", re.MULTILINE)

In [64]:
regex

re.compile(r'(1\s+Projekthintergrund\s+und\s+Zielsetzung\s+4|1\.1\s+Projekthintergrund\s+4|1\.2\s+Die\s+doppelte\s+Zielsetzung\s+des\s+Projekts\s+7|2\s+Konzeption,\s+Planung\s+und\s+Organisation\s+des\s+Dialogprozesses\s+\(AP\s+1\)\s+10|2\.1\s+AP\s+1\s+Erarbeitung\s+des\s+Feinkonzepts\s+10|2\.2\s+Organisation\s+und\s+Prozesssteuerung\s+12|2\.3\s+Qualitätssicherung\s+und\s+Datenschutz\s+13|3\s+Rekrutierung\s+von\s+Zufallsbürger\*innen\s+sowie\s+Eingangs\-\s+und\s+Ausgangsbefragungen\s+14|3\.1\s+Zufallsauswahl\s+der\s+Bürger\*innen\s+\(AP\s+2\)\s+14|3\.2\s+Eingangsbefragung\s+\(AP\s+3\)\s+15|3\.3\s+Ausgangsbefragung\s+und\s+qualitative\s+Erhebung\s+\(AP\s+5\)\s+18|4\s+Durchführung\s+der\s+Werkstätten\s+und\s+Workshops\s+22|4\.1\s+Durchführung\s+der\s+Bürgerwerkstätten\s+\(AP\s+4\)\s+22|4\.2\s+Durchführung\s+des\s+Workshops\s+der\s+Bürgerbotschafter\*innen\s+\(AP\s+6\)\s+24|4\.3\s+Übergabe\s+der\s+Empfehlungen\s+\(AP\s+7\)\s+25|5\s+Berichterstattung\s+27|5\.1\s+Abschlussbericht\s+\(AP\s+8

In [59]:
matches = list(regex.finditer(text))

In [60]:
for i, match in enumerate(matches):
        start = match.start()  # Start from the beginning of the match

        # Define the end position as the start of the next section or the end of the text
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        # Extract the text for this section
        section_text = text[start:end].strip()

        # Store the extracted section text
        extracted_sections[match.group()] = section_text
        print(section_text)

In [53]:
print(extracted_sections)

{}


In [34]:
#Funktion um die extrahierten abschnitte in textdateien zu speichern
def save_sections_as_text_files(extracted_sections, output_directory):
    """
    Saves each section as a separate text file in the specified directory.
    
    :param extracted_sections: Dictionary with section names as keys and content as values.
    :param output_directory: Directory where the text files will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for section, content in extracted_sections.items():
        # Sanitize the section name to make it a valid filename
        filename = re.sub(r'[\\/*?:"<>|]', "_", section)  # Replace invalid characters with underscores
        file_path = os.path.join(output_directory, f"{filename}.txt")

        # Write the content into the file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

        print(f"Section '{section}' saved to '{file_path}'")


output_directory = "extracted_sections"  # Directory to save the text files

# Funktions aufrufen, um die Abschnitte in Textdateien zu speichern
save_sections_as_text_files(extracted_sections, output_directory)